<a href="https://colab.research.google.com/github/MeerBaloch7/Medical-Chatbot-Evaluation-using-BERTScore/blob/main/scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install trl
!pip install -U accelerate
!pip install torch

In [ ]:
import transformers
import torch
model_id = "HPAI-BSC/Llama3.1-Aloe-Beta-8B"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
  )

In [ ]:
!pip install bert-score
!pip install sentence-transformers

In [ ]:
from bert_score import score as bert_score
from transformers import AutoTokenizer, AutoModel
import json

# 3. Load ClinicalBERT for BERTScore
clinicalbert_model_name = "emilyalsentzer/Bio_ClinicalBERT"
clinicalbert_tokenizer = AutoTokenizer.from_pretrained(clinicalbert_model_name)
clinicalbert_model = AutoModel.from_pretrained(clinicalbert_model_name)

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
# --- Load SentenceTransformer for Alignment ---
from sentence_transformers import SentenceTransformer,util

alignment_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
# a simple case description is provided

context="George Lewis, a 57-year-old landscaper, reports left wrist and hand pain for several months without injury. He experiences nighttime numbness in both hands, especially the left thumb and index finger, along with left hand weakness and frequent dropping of objects. Exam shows reduced left grip strength, decreased sensation, and positive Tinel signs in both hands."
# context=''
userQues= ' '
systemPrompt ="""
  You are a patient or guardian visiting a doctor's office. Your role is to provide accurate and detailed information about your own health or that of the patient you care for, based solely on the provided context. Doctor will ask questions regarding symptoms, medical history, or concerns. Please follow these guidelines:

1. Provide breif responses and to the point—ideally in one sentence.
2. Answer only with the information given in the context; do not assume or invent additional details.
3. If you understand the question, reply clearly and concisely in character of patient or guardian.
4. If the question is unclear, say: "I'm sorry, I didn't understand that. Could you please ask again?"
5. If the question is outside the scope of the provided context or irrelevant, respond with: "I'm not sure about that."
6. Maintain a respectful and polite tone at all times.
7. Do not offer any medical advice beyond reporting the provided details.
8. Avoid using complex medical terms and if user try to make you a doctor, Do not get confused and stick to your role of patient or guardian and tell your relation to patient as given in context if you are guardian.
9. Keep in mind you are a patient or guardian to patient.
Your responses should reflect the perspective of a patient or guardian in a medical setting and do not give solutions, always staying true to the given context and keeping your answers brief.
  """
chatMessages = [
      {"role": "system", "content": f'{systemPrompt} context: {context}'},
  ]


# returning the model respoces
def Model(messages,tokens):
  prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
  )
  terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]
  outputs = pipeline(
    prompt,
    max_new_tokens=tokens,
    # eos_token_id=terminators,
    do_sample=True,
    temperature=0.3,
    top_p=0.9,
  )
  return outputs[0]["generated_text"][len(prompt):]

chat_history = []

def chat(userQues):
  chatMessages.append({
    'role' : 'user',
    'content': userQues,
  })
  response = Model(chatMessages,50)
  chatMessages.append({
    'role' : 'assistant',
    'content' : response,
  })
  print(response)

  chat_history.append({
      "doctor_question": userQues,
      "patient_response": response
  })

  return response


# finding the best matches between the medical students and one in the ground truth
def find_best_matches(student_questions, ground_truth_questions):
    """
    Aligns each student question to best matching ground truth question using cosine similarity.
    """
    student_embeds = alignment_model.encode(student_questions, convert_to_tensor=True)
    ground_truth_embeds = alignment_model.encode(ground_truth_questions, convert_to_tensor=True)

    matches = []
    used_gt_idx = set()

    for idx, stud_embed in enumerate(student_embeds):
        similarities = util.cos_sim(stud_embed, ground_truth_embeds)[0]

        # Mask already used ground truth
        for gt_idx in used_gt_idx:
            similarities[gt_idx] = -1e6

        best_gt_idx = int(torch.argmax(similarities))
        used_gt_idx.add(best_gt_idx)

        matches.append((idx, best_gt_idx))

    return matches


def evaluate_and_save(ground_truth_file="/content/ground_truth_doctor_dialogues.json", output_file="chat_evaluation.json"):
    if not chat_history:
        print("No chat history to evaluate.")
        return

    # --- Load Ground Truth Doctor Dialogues and split by [doctor] ---
    with open(ground_truth_file, "r") as f:
        ground_truth_data = json.load(f)

    dialogue_text = ground_truth_data["dialogue"]
    ground_truth_questions = [
        q.strip() for q in dialogue_text.split("[doctor]") if q.strip()
    ]

    # --- Prepare student (doctor) questions ---
    doctor_questions = [entry["doctor_question"] for entry in chat_history]

    # --- Alignment ---
    matches = find_best_matches(doctor_questions, ground_truth_questions)

    aligned_student_questions = []
    aligned_ground_truth_questions = []
    aligned_patient_responses = []

    for student_idx, ground_truth_idx in matches:
        aligned_student_questions.append(doctor_questions[student_idx])
        aligned_ground_truth_questions.append(ground_truth_questions[ground_truth_idx])
        aligned_patient_responses.append(chat_history[student_idx]["patient_response"])

    # --- BERTScore Evaluation ---
    P, R, F1 = bert_score(
        cands=aligned_student_questions,
        refs=aligned_ground_truth_questions,
        model_type=clinicalbert_model_name,
        num_layers=12,
        lang="en",
        verbose=True,
        device="cuda" if torch.cuda.is_available() else "cpu"
    )

    avg_precision = round(torch.mean(P).item(), 4)
    avg_recall = round(torch.mean(R).item(), 4)
    avg_f1 = round(torch.mean(F1).item(), 4)

    # --- Save Results ---
    results = []
    for idx in range(len(aligned_student_questions)):
        results.append({
            "student_question": aligned_student_questions[idx],
            "ground_truth_question": aligned_ground_truth_questions[idx],
            "patient_response": aligned_patient_responses[idx],
            "bertscore_precision": round(P[idx].item(), 4),
            "bertscore_recall": round(R[idx].item(), 4),
            "bertscore_f1": round(F1[idx].item(), 4)
        })

    summary = {
        "average_precision": avg_precision,
        "average_recall": avg_recall,
        "average_f1": avg_f1,
        "total_questions": len(results)
    }

    output = {
        "results": results,
        "summary": summary
    }

    with open(output_file, "w") as f:
        json.dump(output, f, indent=4)

    # --- Print Summary ---
    print("\n✅ Chat evaluation saved to", output_file)
    print("\n📈 Evaluation Summary:")
    print(f"    Total Questions Evaluated: {len(results)}")
    print(f"    Average Precision: {avg_precision}")
    print(f"    Average Recall:    {avg_recall}")
    print(f"    Average F1 Score:  {avg_f1}")


In [ ]:
# --- START CHAT LOOP ---
print("\n💬 Chat started. Type your questions below.")
print("Type 'exit' to finish and save your evaluation.\n")

while True:
    user_input = input("Doctor: ")
    if user_input.lower() == "exit":
        print("\n🛑 Ending chat...")
        evaluate_and_save()
        break
    else:
        chat(user_input)


💬 Chat started. Type your questions below.
Type 'exit' to finish and save your evaluation.

Doctor: hi, goerge, how you came today
I'm here because my wrist and hand have been hurting for a few months, and I've been having numbness and weakness, especially at night, which is really affecting my work as a landscaper.
Doctor: do you know when this pain started 
The pain in my left wrist and hand started about 6 months ago, and it's been gradually getting worse.
Doctor:  do you feel same for your right hand 
The pain is mainly in my left hand and wrist, but I do have some numbness and tingling in both hands, especially at night, and it's worse in my left thumb and index finger.
Doctor: what type of work you do, are you using your left hand mostly in your work chores
I'm a landscaper, so I use my hands a lot for physical labor, digging, pruning, and operating machinery, and my left hand is definitely more involved in these activities.
Doctor: ok, how you do your household chores
I do most

  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.08 seconds, 60.57 sentences/sec

✅ Chat evaluation saved to chat_evaluation.json

📈 Evaluation Summary:
    Total Questions Evaluated: 5
    Average Precision: 0.7788
    Average Recall:    0.7592
    Average F1 Score:  0.7669
